In [1]:
!pip uninstall numpy
!pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2463312 sha256=1eb06fa01455ece9bb453b13e16bec0a49e67df01839813b20360e1e5020e823
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import pandas as pd
from collections import defaultdict
from surprise import Reader, Dataset, SVD, dump, NMF, accuracy, SlopeOne, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate, train_test_split, KFold, GridSearchCV
# from sklearn.model_selection import train_test_split

In [2]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [3]:
reader = Reader(rating_scale=(1,5))

In [4]:
train=pd.read_csv('/content/drive/MyDrive/Kinopoisk/rating_train_df.csv')

In [5]:
param_grid = {"n_epochs": [10, 20, 40], "n_cltr_u": [3, 6, 9], "n_cltr_i": [3, 6, 9]}
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
gs = GridSearchCV(CoClustering, param_grid, measures=["rmse",'mae', "fcp"], cv=10, n_jobs=10, joblib_verbose=5)
gs.fit(data)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed: 15.8min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed: 53.0min
[Parallel(n_jobs=10)]: Done 270 out of 270 | elapsed: 148.5min finished


In [6]:
# best FCP score
print(f'Best FCP score is {gs.best_score["fcp"]}')
# print(gs.best_score["fcp"])

print('Combination of parameters that gave the best FCP score:')
print(gs.best_params["fcp"])

Best FCP score is 0.5484301105503288
Combination of parameters that gave the best FCP score:
{'n_epochs': 40, 'n_cltr_u': 9, 'n_cltr_i': 3}


In [7]:
# # best RMSE score
print(f'Best RMSE score is {gs.best_score["rmse"]}')
# print(gs.best_score["rmse"])

print('Combination of parameters that gave the best RMSE score:')
print(gs.best_params["rmse"])

Best RMSE score is 1.0153735015457561
Combination of parameters that gave the best RMSE score:
{'n_epochs': 20, 'n_cltr_u': 3, 'n_cltr_i': 3}


In [8]:
# best MAE score
print(f'Best MAE score is {gs.best_score["mae"]}')
# print(gs.best_score["mae"])

print('Combination of parameters that gave the best MAE score:')
print(gs.best_params["mae"])

Best MAE score is 0.7820344463722757
Combination of parameters that gave the best MAE score:
{'n_epochs': 20, 'n_cltr_u': 3, 'n_cltr_i': 3}


In [4]:
ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/rating_df.csv', usecols=['userId', 'movieId', 'rating'])

In [5]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [11]:
del ratings

In [12]:
del reader

In [6]:
trainset, testset = train_test_split(data, train_size=0.8, test_size=0.2)

In [14]:
del data

In [7]:
coclustering = CoClustering(n_epochs=20, n_cltr_u = 3, n_cltr_i = 3)
coclustering.fit(trainset)
del trainset

In [8]:
predictions = coclustering.test(testset)


In [17]:
print(f"CoClustering model's RMSE {accuracy.rmse(predictions)}")
print(f"CoClustering model's MAE {accuracy.mae(predictions)}")
print(f"CoClustering model's FCP {accuracy.fcp(predictions)}")

RMSE: 0.8958
CoClustering model's RMSE 0.8957521614863706
MAE:  0.6934
CoClustering model's MAE 0.6933679595194947
FCP:  0.6700
CoClustering model's FCP 0.6700448320512495


In [9]:
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)

# Precision and recall can then be averaged over all users
print(f"CoClustering model's precision at 10 {sum(prec for prec in precisions.values()) / len(precisions)}")
print(f"CoClustering model's recall at 10 {sum(rec for rec in recalls.values()) / len(recalls)}")

CoClustering model's precision at 10 0.7061062204918657
CoClustering model's recall at 10 0.5694414635239082
